In [0]:
%run ./IMPORTS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 42.7 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
#create table s_sensor_iot
spark.sql("""
    CREATE TABLE IF NOT EXISTS s_sensor_iot (
        dt_refe STRING,
        qt_corr INT,
        qt_corr_neg INT,
        qt_corr_pess INT,
        vl_max_dist DOUBLE,
        vl_min_dist DOUBLE,
        vl_avg_dist DOUBLE,
        qt_corr_reuni INT,
        qt_corr_nao_reuni INT,
        dt_evento TIMESTAMP
    )
    USING delta
""")
print("Tabela criada com sucesso s_sensor_iot")

Tabela criada com sucesso s_sensor_iot


#PRODUCER

In [0]:

fake = Faker()

# Define o schema
schema = StructType([
    StructField("sensor_id", StringType(), True),
    StructField("timestamp", TimestampType(), True),
    StructField("temperatura", DoubleType(), True),
    StructField("umidade", DoubleType(), True),
    StructField("latitude", StringType(), True)
])

# função criada para sensor data
def gerador_sensor_data():
    return {
        "sensor_id": str(fake.uuid4()),
        "timestamp": datetime.now(),  # Keep as datetime object
        "temperatura": random.uniform(20, 45),
        "umidade": random.uniform(30, 90),
        "latitude": str(fake.latitude())
    }

# # range(n) - gera a quantidade de lotes para mostar na tela
for i in range(2):
    df_lote = spark.createDataFrame([gerador_sensor_data() for _ in range(10)], schema=schema)
    df_stream = df_lote.union(df_lote)
    print(f"Lote {i+1} adicionado com sucesso")
    df_lote.show()
    time.sleep(3)
    clear_output(wait=True)

Lote 2 adicionado com sucesso
+--------------------+--------------------+------------------+------------------+-----------+
|           sensor_id|           timestamp|       temperatura|           umidade|   latitude|
+--------------------+--------------------+------------------+------------------+-----------+
|574ac9fb-386f-471...|2025-07-22 23:27:...|22.719916366913697| 32.87358920821814|-52.6132355|
|27cb4e9f-439b-4e6...|2025-07-22 23:27:...| 24.02924070404241| 71.95983726304391|  25.254700|
|6542cd3b-1a88-4b5...|2025-07-22 23:27:...|28.778724143556275| 71.68298766657486|  9.4668905|
|9e887b81-8290-4e7...|2025-07-22 23:27:...|27.616884598964862| 73.03918003067032|-59.6801355|
|4abc7d21-b387-45d...|2025-07-22 23:27:...|  39.4769727678473| 67.58049827635685|-20.7874815|
|ac7d1870-6e4e-4a0...|2025-07-22 23:27:...|41.509335806390546| 48.79750534223887|  39.720846|
|f634ecb3-2c49-45c...|2025-07-22 23:27:...| 22.17201212860838| 31.42209178564039|  65.400651|
|e38899dd-61e9-425...|2025-07-

#CONSUMER

In [0]:
df_create = df_stream.withColumn(
    "dt_refe", date_format(col("timestamp"), "yyyy-MM-dd")
).select(
    col("sensor_id"),
    round(col("temperatura"),2).alias("temperature"),
    round(col("umidade"),2).alias("umidade"),
    col("latitude"),
    col("dt_refe"),
)

df_create.write.mode("append").format("delta").saveAsTable("s_sensor_iot")